In [73]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [74]:
import numpy as np
import sys
LIBS_AI = '/home/ubuntu/fastai/'
sys.path.append(LIBS_AI)

In [75]:
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

In [76]:
DATA_PATH_EN = '../data/comtrans_en.txt'
DATA_PATH_DE = '../data/comtrans_en.txt'

In [78]:
raw_data = open(DATA_PATH_EN).read()

In [79]:
print('corpus length:', len(raw_data))

corpus length: 3885310


In [80]:
raw_data[:400]

"Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Friday 17 December 1999 , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .\nAlthough , as you will have seen , the dreaded ' millennium bug ' failed to materialise , still the people in a number of countries suffered a series of natural dis"

In [81]:
chars = sorted(list(set(raw_data)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 83


In [82]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [83]:
cs=8
idx = [char_indices[c] for c in raw_data]
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

In [84]:
''.join(chars)

'\n !"$%&\'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz'

In [180]:
target = '.,?!#'

# Traget Encodings
char_indices_t = {c: i for i, c in enumerate(target)}
indices_char_t = {i: c for i, c in enumerate(target)}

target_enc = []

for i in c_in_dat:
    target_line = [0,0,0,0,0]
    for k,v in enumerate(i):
        current_char = indices_char[v]
        if current_char in char_indices_t:
            target_line[char_indices_t[current_char]] = k 
        else:
            target_line[4] = 1
    target_enc.append(target_line)

In [121]:
xs = np.stack(c_in_dat, axis=0)

In [122]:
y = np.stack(target_enc)

In [123]:
y

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [8, 0, 0, 0, 1]])

In [124]:
xs

array([[45, 60, 74, ..., 71, 75, 64],
       [60, 74, 76, ..., 75, 64, 70],
       [74, 76, 68, ..., 64, 70, 69],
       ...,
       [80,  1, 73, ..., 63, 75, 74],
       [ 1, 73, 64, ..., 75, 74,  1],
       [73, 64, 62, ..., 74,  1, 13]])

In [125]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [156]:
n_fac = 30; n_hidden = 10; bs =128; nl=2

In [173]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp), dim=-1)

In [175]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y)

In [176]:
m = GruRNN(vocab_size, n_fac, 7).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [177]:
fit(m, md, 5, opt, F.cross_entropy)

  0%|          | 0/48567 [00:00<?, ?it/s]


TypeError: forward() takes 3 positional arguments but 9 were given